In [1]:
# !jupyter nbconvert ollama-with-openai.ipynb --to python --output ../src/smart_procurement/models/chat_interface.py

In [1]:
from llama_index.llms.ollama import Ollama

In [2]:
llm = Ollama(model="gemma3:4b", request_timeout=120.0)

In [3]:
SYSTEM_PROMPT = """You are a helpful assistant designed to match user queries with relevant commodity codes with descriptions as context used in procurement purchase orders.
You will go through the description provided for each commodity code and try to match the user's query with atleast top 3 relevant commodity codes.
You must always respond with a valid JSON array of objects where each object contains:
1. "code": The ID of the commodity code (as a string).
2. "confidence": A number from 0 to 100 indicating your confidence that the commodity code matches the user's query.
3. code and confidence both should be integers.
4. You should go through all the descriptions and then decide which commodity code matches the user's query.

The JSON format should look like this wrapped with markdown code block:
```json
[
    {"code": 10001, "confidence": 80},
    {"code": 10002, "confidence": 60},
    ...
]
```
Only include the commodity codes that have a relevance or confidence above 30. 
If no commodity code matches the query, return an empty array.

Always ensure the JSON response is valid.
After you prepared the json you must validate it that is a valid json and return it to user.
Provide me only the JSON and nothing else. Do not add any extra text in the beginning or end of the JSON.
"""

In [4]:
CONTEXT = """
1. 10001 - Computer, Hardware
2. 10002 - Software for Computer
3. 10003 - Office Furniture
4. 10004 - Computer Accessories
5. 10005 - Office Supplies
"""

QUERY = "I want to buy a keyboard and mouse."

In [5]:
USER_PROMPT = f"""Query:\n{QUERY} \n\n Context:\n{CONTEXT}\n"""

In [6]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", 
        content=SYSTEM_PROMPT,
    ),
    ChatMessage(role="user", content=USER_PROMPT),
]

In [7]:
resp = llm.chat(messages)

In [8]:
resp.message.content

'```json\n[\n    {"code": 10004, "confidence": 95},\n    {"code": 10001, "confidence": 60},\n    {"code": 10005, "confidence": 30}\n]\n```'

In [9]:
import json
import re

def extract_json_from_markdown(markdown_content):
    # Regular expression to find JSON content between markdown code block
    json_pattern = r"```json\n(.*?)\n```"
    match = re.search(json_pattern, markdown_content, re.DOTALL)
    
    if match:
        json_content = match.group(1)
        try:
            # Parse and return the JSON object
            return json.loads(json_content)
        except json.JSONDecodeError:
            return None
    return None


In [10]:
extract_json_from_markdown(resp.message.content)

[{'code': 10004, 'confidence': 95},
 {'code': 10001, 'confidence': 60},
 {'code': 10005, 'confidence': 30}]

In [62]:
from pydantic import BaseModel

In [63]:
class CommodityCodeResult(BaseModel):
    code: int
    confidence: int


In [64]:
class CommodityCodeResultList(BaseModel):
    results: list[CommodityCodeResult]

    def get_all_codes(self) -> list[int]:
        if self.results:
            return [int(item.code) for item in self.results]
        return []
    
    @classmethod
    def from_model_response_str(cls, response_str: str) -> Self:
        response_dict = extract_json_from_markdown(response_str)
        if response_dict:
            return cls(results=[CommodityCodeResult(**item) for item in response_dict])
        return cls(results=[])
    
    def get_key_value_pairs(self) -> dict[int, int]:
        return {item.code: item.confidence for item in self.results}

In [65]:
commodit_codes = CommodityCodeResultList.from_model_response_str(resp.message.content)

In [66]:
commodit_codes.get_all_codes()

[10004, 10005]

In [11]:
from enum import StrEnum


class ModelSelection(StrEnum):
    """Enum for selecting the available LLM models."""
    GEMMA3 = "gemma3:4b"
    DEEPSEEKR1 = "deepseek-r1:7b"
    NOMIC = "nomic-embed-text:v1.5"


In [12]:
CONTEXT = """
PharmaCorp, a global pharmaceutical giant, has finalized a landmark Master Level Agreement 
with ProcureMe, a leading provider of intelligent procurement solutions leveraging SAP Ariba 
Contracts. The agreement marks a significant step in PharmaCorp’s digital transformation 
strategy, aimed at streamlining its sourcing processes and accelerating innovation. Under the 
terms of the agreement, PharmaCorp will be outsourcing the development and maintenance of its 
Gen AI-powered procurement tools to a dedicated team of 3 Gen AI developers staffed by 
ProcureMe. This partnership represents a commitment to embracing cutting-edge technology and 
optimizing operational efficiency.

The core of the agreement revolves around ProcureMe's expertise in utilizing SAP Ariba 
Contracts, an established platform adopted by numerous Fortune 500 companies. The Gen AI 
developers, working alongside PharmaCorp’s procurement team, will focus on customizing the 
Ariba Contracts platform with Gen AI capabilities. This includes developing automated contract 
review tools, intelligent spend analysis, and proactive risk mitigation strategies. The 
initial phase of the project will concentrate on optimizing PharmaCorp’s supplier onboarding 
process, significantly reducing administrative overhead.

ProcureMe’s selection was driven by the company’s deep understanding of large enterprise 
procurement workflows and its proven track record of delivering impactful Gen AI solutions. 
The partnership integrates seamlessly with PharmaCorp’s existing IT infrastructure and 
utilizes a secure, cloud-based environment, ensuring data privacy and compliance with industry 
regulations. Furthermore, the contract includes robust Service Level Agreements (SLAs) 
guaranteeing performance, uptime, and ongoing support.

Both companies view this collaboration as a catalyst for future innovation.  PharmaCorp 
anticipates that the Gen AI-powered solutions will not only reduce operational costs but also 
empower its procurement team to make more data-driven decisions.  ProcureMe, in turn, gains 
valuable exposure to a complex, regulated industry and an opportunity to refine its Gen AI 
offerings for large-scale enterprise deployments. The long-term goal is to foster a strategic 
relationship built on continuous improvement and collaborative innovation.
"""

In [35]:
QUESTIONS = [
    "What is the master level agreement between PharmaCorp and ProcureMe?",
    "What are the key features of the Gen AI-powered procurement tools?",
    "How does PharmaCorp optimize its supplier onboarding process?",
    "What are the SLAs included in the contract?",
    "What are the long-term goals of the partnership?",
    "What abc company is buying from DEF company?"
]

In [54]:
SYSTEM_PROMPT_AUGMENTED_GENERATION = """
You are a helpful and informative assistant powered by a Retrieval-Augmented Generation (RAG) 
system. Your primary goal is to answer user questions accurately and concisely, leveraging a 
provided context document. You will output your responses in JSON format.

**Instructions:**

1.  **Receive Context:** You will be provided with a document (referred to as 
"[CONTEXT_DOCUMENT_NAME]") containing relevant information. *Carefully read and understand 
this document.*

2.  **Receive Query:** You will also receive a user's question (referred to as 
"[USER_QUERY]").

3.  **Retrieve Relevant Information:** Based on the [USER_QUERY], identify the most relevant 
segments of the [CONTEXT_DOCUMENT_NAME]. Focus on extracting direct answers and supporting 
details.

4.  **Generate Answer:** Synthesize the retrieved information into a clear, concise, and 
accurate answer to the [USER_QUERY].

5.  **Output JSON Response:** Format your response as a JSON object with the following 
structure:

    ```json
    {
      "response": "[Generated Answer with Citation Numbering]",
      "sources": [
        {
          "document_title": "[CONTEXT_DOCUMENT_NAME]",
          "section_text": "[Relevant Section/Paragraph Text]"
        }
      ]
    }
    ```

    *   `response`:  The generated answer to the user's question, including the citation 
number. Example: “The agreement assigns 3 Gen AI developers. [1]”
    *   `sources`: An array containing details about the source of the information. Each 
object in the array should include:
        *   `document_title`: The title of the document where the answer was found.
        *   `section_text`:  The exact text from the document that supports the answer.
        *   `success`: true/false based on either answered the question or not

6.  **Maintain Professional Tone:** Respond in a professional and informative tone.

7. **If the answer cannot be found within the context document, respond with:** “I’m sorry, I 
cannot answer that question based on the provided context document.” 

8. ** the answer cannot be found within the context:
   - `sources` should be empty list: Example: sources: []
   - success should be false Example: success: false
   - No numbering will be added into the response

**Example:**

Success Example:
*   [CONTEXT_DOCUMENT_NAME]: PharmaCorp and ProcureMe Agreement
*   [USER_QUERY]: How many Gen AI developers will ProcureMe be assigning to PharmaCorp?

*   Response:
    ```json
    {
      "response": "The agreement assigns 3 Gen AI developers. [1]",
      "success": true,
      "sources": [
        {
          "document_title": "PharmaCorp and ProcureMe Agreement",
          "section_text": "ProcureMe will be assigning 3 Gen AI developers to support 
PharmaCorp’s procurement needs."
        }
      ]
    }
    ```
Fail Example:
*   [CONTEXT_DOCUMENT_NAME]: PharmaCorp and ProcureMe Agreement
*   [USER_QUERY]: How many Gen AI developers will LargeCorp be assigning to XCorp?
  ``json
      {
        "response": "I’m sorry, I cannot answer that question based on the provided context document.",
        "success": false,
        "sources": []
      }
  ```

**Important:** Always adhere to the specified JSON format.
"""

In [55]:
QUESTIONS

['What is the master level agreement between PharmaCorp and ProcureMe?',
 'What are the key features of the Gen AI-powered procurement tools?',
 'How does PharmaCorp optimize its supplier onboarding process?',
 'What are the SLAs included in the contract?',
 'What are the long-term goals of the partnership?',
 'What abc company is buying from DEF company?']

In [56]:
USER_PROMPT = f"""Query:\n{QUESTIONS[5]} \n\n Context:\n{CONTEXT}\n"""

In [57]:
messages = [
    ChatMessage(
        role="system", 
        content=SYSTEM_PROMPT_AUGMENTED_GENERATION,
    ),
    ChatMessage(role="user", content=USER_PROMPT),
]

In [58]:
resp = llm.chat(messages)

In [59]:
resp.message.content

'```json\n{\n  "response": "The document does not specify which company ABC is buying from DEF company.",\n  "success": false,\n  "sources": []\n}\n```'

In [3]:
import ollama

In [13]:


def main():
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me a short story in 3 lines."},
        {"role": "assistant", "content": "Here is 1 line. Here is line 2. Here is line 3."},
        {"role": "user", "content": "The story you told me was is good enough? Yes or no?"},
    ]
    response = ollama.chat(
        messages=messages,
        model="gemma3:4b",
        stream=True
    )
    # Process the response as needed
    for chunk in response:
        content = chunk['message']['content']
        print(content, end='', flush=True)
    
    return response

In [15]:
# main()